In [1]:
import torch

In [2]:
import numpy as np

In [12]:
ent = [1,0,0,0,0] # entrance
nmr = [0,1,0,0,0] # normal room
bthr = [0,0,1,0,0] # bath room
ktc = [0,0,0,1,0] # kitchen
lvr = [0,0,0,0,1] # living room